- recuperer la donner 
- faire un fonction pour recuperer la donnée 
- nettoyer la donnée
- fonction de netoyage 

In [50]:
import requests
import json
import requests
import json
import pandas as pd
from typing import Optional


In [74]:

def find(query: str) -> Optional[pd.DataFrame]:
    """
    Finds places based on the query string using the Google Places API.

    Args:
        query (str): The search query to find places.

    Returns:
        Optional[pd.DataFrame]: A DataFrame containing the places data, or None if an error occurs.
    """
    with open('api.txt') as file:
        api_key = file.read().strip()  # Remove extra whitespace or newlines

    url = "https://places.googleapis.com/v1/places:searchText"

    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": api_key,
        "X-Goog-FieldMask": "places.displayName,places.formattedAddress,places.location,places.rating,places.userRatingCount,places.currentOpeningHours,places.reviews,places.internationalPhoneNumber,places.websiteUri,places.photos,places.location,places.priceLevel"
    }

    data = {
        "textQuery": query
    }

    try:

        response = requests.post(url, headers=headers, data=json.dumps(data))
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx and 5xx)

        response_json = response.json()
        places = response_json.get('places', [])  
        if places:
            df = pd.json_normalize(places)
            return df
        else:
            print("No places found in the response.")
            return None

    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return None

    except json.JSONDecodeError as e:
        print(f"Failed to parse JSON response: {e}")
        return None


In [ ]:
find('manger un hamburger')

,internationalPhoneNumber,formattedAddress,rating,websiteUri,priceLevel,userRatingCount,reviews,photos,location.latitude,location.longitude,displayName.text,displayName.languageCode,currentOpeningHours.openNow,currentOpeningHours.periods,currentOpeningHours.weekdayDescriptions,currentOpeningHours.nextOpenTime
0,+33 5 61 14 01 40,"54 Av. Jules Julien, 31400 Toulouse, France",4.3,https://pimpmyburger.fr/,PRICE_LEVEL_MODERATE,866,[{'name': 'places/ChIJudjYhHG8rhIRENfRNnNoe28/...,[{'name': 'places/ChIJudjYhHG8rhIRENfRNnNoe28/...,43.576136,1.451329,Pimp My Burger,fr,False,"[{'open': {'day': 0, 'hour': 11, 'minute': 30,...","[Monday: 11:30 AM – 1:45 PM, 6:30 – 10:00 PM, ...",2025-01-18T10:30:00Z


In [101]:
import json
import requests
import pandas as pd
from typing import Optional
import time

def findv2(query: str) -> Optional[pd.DataFrame]:
    """
    Finds places based on the query string using the Google Places API.

    Args:
        query (str): The search query to find places.

    Returns:
        Optional[pd.DataFrame]: A DataFrame containing the places data, or None if an error occurs.
    """
    try:
        with open('api.txt') as file:
            api_key = file.read().strip()
    except FileNotFoundError:
        print("Error: API key file 'api.txt' not found.")
        return None

    url = "https://places.googleapis.com/v1/places:searchText"
    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": api_key,
        "X-Goog-FieldMask": "places.displayName,places.formattedAddress,places.location,places.rating,places.userRatingCount,places.currentOpeningHours,places.reviews,places.internationalPhoneNumber,places.websiteUri,places.photos,places.location,places.priceLevel"
    }
    data = {"textQuery": query}

    results = []  # Liste pour stocker tous les résultats
    print(results)
    while True:
        try:
            response = requests.post(url, headers=headers, data=json.dumps(data))
            response.raise_for_status()

            response_json = response.json()
            places = response_json.get('places', [])
            results.extend(places)

            # Vérifiez si un nextPageToken existe
            next_page_token = response_json.get('nextPageToken')
            if not next_page_token:  # Pas de page suivante, arrêter la boucle
                break

            # Ajouter une pause avant la prochaine requête
            time.sleep(2)  # Délai pour attendre que le nextPageToken devienne actif
            data['pageToken'] = next_page_token

        except requests.exceptions.RequestException as e:
            print(f"Request failed: {e}")
            return None
        except json.JSONDecodeError as e:
            print(f"Failed to parse JSON response: {e}")
            return None

    if results:
        df = pd.json_normalize(results)
        return df
    else:
        print("No places found in the response.")
        return None


In [104]:
test = findv2('chinois')

[]


In [105]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 17 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   internationalPhoneNumber                 20 non-null     object 
 1   formattedAddress                         20 non-null     object 
 2   rating                                   20 non-null     float64
 3   websiteUri                               16 non-null     object 
 4   userRatingCount                          20 non-null     int64  
 5   reviews                                  20 non-null     object 
 6   photos                                   20 non-null     object 
 7   location.latitude                        20 non-null     float64
 8   location.longitude                       20 non-null     float64
 9   displayName.text                         20 non-null     object 
 10  displayName.languageCode                 20 non-null

In [107]:
test

,internationalPhoneNumber,formattedAddress,rating,websiteUri,userRatingCount,reviews,photos,location.latitude,location.longitude,displayName.text,displayName.languageCode,currentOpeningHours.openNow,currentOpeningHours.periods,currentOpeningHours.weekdayDescriptions,currentOpeningHours.nextOpenTime,priceLevel,currentOpeningHours.nextCloseTime
0,+33 5 61 73 06 05,"24 Pl. Marnac, 31520 Ramonville-Saint-Agne, Fr...",4.7,http://www.etape-d-asie.fr/,391,[{'name': 'places/ChIJ7VfBSYG-rhIRkZvVIXFi8gg/...,[{'name': 'places/ChIJ7VfBSYG-rhIRkZvVIXFi8gg/...,43.546611,1.478304,Etape D'Asie,fr,False,"[{'open': {'day': 1, 'hour': 12, 'minute': 0, ...","[Monday: 12:00 – 2:00 PM, 7:00 – 10:00 PM, Tue...",2025-01-18T18:00:00Z,NaN,NaN
1,+33 9 54 52 48 19,"44 Bd Lazare Carnot, 31000 Toulouse, France",4.3,https://lesthermes-toulouse.fr/,842,[{'name': 'places/ChIJc_GIv7C9rhIRoc6DCHiCEeY/...,[{'name': 'places/ChIJc_GIv7C9rhIRoc6DCHiCEeY/...,43.604551,1.450263,Les Thermes Restaurant Chinois,fr,True,"[{'open': {'day': 0, 'hour': 7, 'minute': 0, '...","[Monday: 7:00 AM – 1:00 AM, Tuesday: 7:00 AM –...",NaN,PRICE_LEVEL_MODERATE,2025-01-18T01:00:00Z
2,+33 5 61 75 94 88,"1 All. des Minimes, 31320 Auzeville-Tolosane, ...",4.3,http://aux-delices.com/,578,[{'name': 'places/ChIJY6KAiFy-rhIRs7E3SghUtM4/...,[{'name': 'places/ChIJY6KAiFy-rhIRs7E3SghUtM4/...,43.527425,1.490088,Aux Delices d'Asie,fr,False,"[{'open': {'day': 0, 'hour': 12, 'minute': 0, ...","[Monday: 12:00 – 2:00 PM, 7:00 – 10:00 PM, Tue...",2025-01-18T11:00:00Z,PRICE_LEVEL_INEXPENSIVE,NaN
3,+33 5 62 48 24 50,"281 Rte d'Espagne, 31100 Toulouse, France",4.2,https://www.royalbuffet.fr/,2575,[{'name': 'places/ChIJaaYgiqm5rhIRrsN8e-MwZcM/...,[{'name': 'places/ChIJaaYgiqm5rhIRrsN8e-MwZcM/...,43.549866,1.421048,Royal Buffet Toulouse Route d'Espagne,fr,False,"[{'open': {'day': 0, 'hour': 12, 'minute': 0, ...","[Monday: 12:00 – 2:30 PM, 7:00 – 11:00 PM, Tue...",2025-01-18T11:00:00Z,PRICE_LEVEL_MODERATE,NaN
4,+33 5 61 39 07 17,"362 Rte de Baziege la Lauragaise, 31670 Labège...",4.2,http://www.lesasiatides.com/,503,[{'name': 'places/ChIJeXudJ_iVrhIRH1G0Zw3p2Ew/...,[{'name': 'places/ChIJeXudJ_iVrhIRH1G0Zw3p2Ew/...,43.548661,1.512370,Les Asiatides,fr,False,"[{'open': {'day': 1, 'hour': 12, 'minute': 0, ...","[Monday: 12:00 – 2:00 PM, 7:00 – 10:00 PM, Tue...",2025-01-18T11:00:00Z,PRICE_LEVEL_INEXPENSIVE,NaN
5,+33 5 61 39 90 00,"43 Av. de Toulouse, 31650 Saint-Orens-de-Gamev...",4.2,http://www.citemandarin.com/,689,[{'name': 'places/ChIJx3vtlgSWrhIRjwQSF3Y0nNE/...,[{'name': 'places/ChIJx3vtlgSWrhIRjwQSF3Y0nNE/...,43.558857,1.526628,La Cité Du Mandarin,fr,False,"[{'open': {'day': 0, 'hour': 12, 'minute': 0, ...","[Monday: Closed, Tuesday: 12:00 – 2:00 PM, 7:0...",2025-01-18T11:00:00Z,PRICE_LEVEL_MODERATE,NaN
6,+33 9 85 04 41 42,"286 Rte de Revel, 31400 Toulouse, France",3.9,https://www.facebook.com/ledynastie31400,1124,[{'name': 'places/ChIJf4sY8u29rhIR0HrMap5Wd7c/...,[{'name': 'places/ChIJf4sY8u29rhIR0HrMap5Wd7c/...,43.565576,1.510926,Le Dynastie,fr,False,"[{'open': {'day': 0, 'hour': 12, 'minute': 0, ...","[Monday: 12:00 – 2:30 PM, 7:00 – 10:30 PM, Tue...",2025-01-18T11:00:00Z,PRICE_LEVEL_MODERATE,NaN
7,+33 9 54 31 56 64,"42 Rue Peyrolières, 31000 Toulouse, France",4.6,NaN,435,[{'name': 'places/ChIJ17a5TmK7rhIRlC_iesYBHxM/...,[{'name': 'places/ChIJ17a5TmK7rhIRlC_iesYBHxM/...,43.601568,1.440784,Restaurant Long Quan,fr,False,"[{'open': {'day': 2, 'hour': 12, 'minute': 0, ...","[Monday: Closed, Tuesday: 12:00 – 2:00 PM, 7:0...",2025-01-18T11:00:00Z,PRICE_LEVEL_MODERATE,NaN
8,+33 5 34 24 20 89,"13 Rue du Rem Villeneuve, 31000 Toulouse, France",4.9,https://delicity.com/jiao-zi-guan-toulouse,725,[{'name': 'places/ChIJv1Y0TKm9rhIRXJ0bXygYL7s/...,[{'name': 'places/ChIJv1Y0TKm9rhIRXJ0bXygYL7s/...,43.605800,1.446345,Jiao zi guan (l'art de ravioli),fr,False,"[{'open': {'day': 1, 'hour': 12, 'minute': 0, ...","[Monday: 12:00 – 2:30 PM, 7:00 – 10:30 PM, Tue...",2025-01-18T11:00:00Z,PRICE_LEVEL_MODERATE,NaN
9,+33 5 61 16 68